In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

import sklearn.metrics as skm

import time

In [2]:
data = np.loadtxt(open('../00-Dataset/mnist_train_small.csv', 'rb'), delimiter=',')

In [3]:
# extract labels and features
labels = data[:,0]
data = data[:,1:]
# normalize the data to a range of [0, 1]
dataNorm = data/np.max(data)

In [4]:
# Convert to tensor
dataT = torch.Tensor(dataNorm).float()
labelsT = torch.Tensor(labels).long()

In [5]:
# split the data
train_data, test_data, train_label, test_label = train_test_split(dataT, labelsT, train_size=0.9)
train_dataT = TensorDataset(train_data, train_label)
test_dataT = TensorDataset(test_data, test_label)

train_loader = DataLoader(train_dataT, batch_size=32, drop_last=True, shuffle=True)
test_loader = DataLoader(test_dataT, batch_size=test_dataT.tensors[0].shape[0])

In [6]:
class Model(nn.Module):
    def __init__ (self):
        super().__init__()

        self.input = nn.Linear(784, 64)
        self.h1 = nn.Linear(64, 32)
        self.h2 = nn.Linear(32, 32)
        self.output = nn.Linear(32, 10)

    def forward(self, data):
        x = F.relu(self.input(data))
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        x = self.output(x)
        # x = torch.log_softmax(self.output(x), axis=1)
        # log_softmax() because of NLLLoss instead of CrossEntropyLoss

        return x

In [7]:
def getModel():
    ANN = Model()
    lossFunction = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=ANN.parameters(), lr=0.01)

    return ANN, lossFunction, optimizer

In [8]:
def trainModel(ANN, lossFunction, optimizer):

    trainingStartTime = time.process_time()

    epochs = 10

    losses = torch.zeros(epochs)
    trainAcc = []
    testAcc = []

    for epochi in range(epochs):
        batchAcc = []
        batchLoss = []
        for x, y in train_loader:
            yHat = ANN.forward(x)
            
            loss = lossFunction(yHat, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batchLoss.append(loss.item())
            batchAcc.append(torch.mean((torch.argmax(yHat, axis=1)==y).float())*100)

        trainAcc.append(np.mean(batchAcc))
        losses[epochi] = np.mean(batchLoss)

        X, Y = next(iter(test_loader))

        with torch.no_grad():
            pred = ANN(X)

        testAcc.append(torch.mean((torch.argmax(pred, axis=1)==Y).float())*100)

        # Finally, report the epoch number, computation time, and accuracy
        comptime = time.process_time() - trainingStartTime
        print(f'Epoch {epochi+1}/{epochs}, elapsed time: {comptime:.2f} sec, test accuracy {testAcc[-1]:.0f}%')

    return trainAcc, testAcc, losses, ANN

In [9]:
ANN, lossFunction, optimizer = getModel()
trainAcc, testAcc, losses, ANN = trainModel(ANN, lossFunction, optimizer)

Epoch 1/10, elapsed time: 4.14 sec, test accuracy 89%
Epoch 2/10, elapsed time: 7.74 sec, test accuracy 94%
Epoch 3/10, elapsed time: 11.64 sec, test accuracy 94%
Epoch 4/10, elapsed time: 16.07 sec, test accuracy 93%
Epoch 5/10, elapsed time: 20.35 sec, test accuracy 95%
Epoch 6/10, elapsed time: 24.69 sec, test accuracy 95%
Epoch 7/10, elapsed time: 28.99 sec, test accuracy 95%
Epoch 8/10, elapsed time: 33.15 sec, test accuracy 94%
Epoch 9/10, elapsed time: 37.43 sec, test accuracy 94%
Epoch 10/10, elapsed time: 41.76 sec, test accuracy 95%
